# Parametric Curve Parameter Estimation

Interactive analysis notebook for exploring the optimization results.

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from optimizer import ParametricCurveOptimizer, load_data
from visualizer import generate_plots

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries imported successfully!")

## 1. Load Data

In [ ]:
# Load the data
data = load_data('../data/xy_data.csv')

# Display basic information
print(f"Number of data points: {len(data)}")
print(f"\nFirst few rows:")
print(data.head())

print(f"\nData statistics:")
print(data.describe())

## 2. Visualize Input Data

In [ ]:
# Plot the raw data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(data['x'], data['y'], alpha=0.5, s=10)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title('Observed Data Points', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Histogram
axes[1].hist(data['x'], bins=30, alpha=0.5, label='x', color='blue')
axes[1].hist(data['y'], bins=30, alpha=0.5, label='y', color='red')
axes[1].set_xlabel('Value', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Data Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Run Optimization

In [ ]:
# Create optimizer
optimizer = ParametricCurveOptimizer(
    theta_range=(0, 50),
    M_range=(-0.05, 0.05),
    X_range=(0, 100)
)

# Fit parameters (this will take some time)
print("Starting optimization...")
result = optimizer.fit(data)

print("\nOptimization complete!")

## 4. Display Results

In [ ]:
# Display optimal parameters
print("=" * 60)
print("OPTIMAL PARAMETERS")
print("=" * 60)
print(f"θ (theta) = {result['theta']:.10f}°")
print(f"M         = {result['M']:.12f}")
print(f"X         = {result['X']:.10f}")
print(f"\nL1 Distance = {result['L1_distance']:.10f}")
print(f"Time Elapsed = {result['time_elapsed']:.2f} seconds")
print("=" * 60)

# Generate Desmos format
desmos_format = optimizer.generate_desmos_format()
print("\nDesmos Format:")
print(desmos_format)

## 5. Visualize Fitted Curve

In [ ]:
# Generate fine curve for visualization
theta, M, X = optimizer.best_params
t_fine = np.linspace(6, 60, 1000)
x_pred, y_pred = optimizer.compute_curve(theta, M, X, t_fine)

# Plot
fig, ax = plt.subplots(figsize=(14, 10))

# Observed data
ax.scatter(data['x'], data['y'], c='red', s=20, alpha=0.5, 
          label='Observed Data', zorder=3)

# Fitted curve
ax.plot(x_pred, y_pred, 'b-', linewidth=2, 
       label='Fitted Curve', zorder=2)

ax.set_xlabel('x', fontsize=14, fontweight='bold')
ax.set_ylabel('y', fontsize=14, fontweight='bold')
ax.set_title('Parametric Curve Fitting Results', 
            fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Analyze Residuals

In [ ]:
# Calculate residuals
t_data = np.linspace(6, 60, len(data))
x_pred_data, y_pred_data = optimizer.compute_curve(theta, M, X, t_data)

x_residuals = data['x'].values - x_pred_data
y_residuals = data['y'].values - y_pred_data
total_residuals = np.abs(x_residuals) + np.abs(y_residuals)

# Plot residuals
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# X residuals
axes[0, 0].scatter(t_data, x_residuals, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('t')
axes[0, 0].set_ylabel('X Residual')
axes[0, 0].set_title('X-Coordinate Residuals')
axes[0, 0].grid(True, alpha=0.3)

# Y residuals
axes[0, 1].scatter(t_data, y_residuals, alpha=0.5, color='green')
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('t')
axes[0, 1].set_ylabel('Y Residual')
axes[0, 1].set_title('Y-Coordinate Residuals')
axes[0, 1].grid(True, alpha=0.3)

# Histogram
axes[1, 0].hist(total_residuals, bins=50, color='purple', alpha=0.7)
axes[1, 0].set_xlabel('L1 Distance per Point')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Residual Distribution')
axes[1, 0].grid(True, alpha=0.3)

# Statistics
axes[1, 1].axis('off')
stats_text = f"""Residual Statistics:

Mean L1: {np.mean(total_residuals):.6f}
Std L1: {np.std(total_residuals):.6f}
Max L1: {np.max(total_residuals):.6f}
Min L1: {np.min(total_residuals):.6f}
Median L1: {np.median(total_residuals):.6f}

Mean X Residual: {np.mean(x_residuals):.6f}
Mean Y Residual: {np.mean(y_residuals):.6f}
"""
axes[1, 1].text(0.1, 0.5, stats_text, fontsize=12, 
               verticalalignment='center', family='monospace')

plt.tight_layout()
plt.show()

## 7. Optimization Convergence

In [ ]:
# Plot convergence history
if optimizer.optimization_history:
    stages = [h['stage'] for h in optimizer.optimization_history]
    distances = [h['distance'] for h in optimizer.optimization_history]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(range(len(stages)), distances, 'o-', linewidth=2, markersize=10)
    ax.set_xlabel('Optimization Stage', fontsize=12)
    ax.set_ylabel('L1 Distance', fontsize=12)
    ax.set_title('Optimization Convergence', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(stages)))
    ax.set_xticklabels([s.replace('_', ' ').title() for s in stages], 
                       rotation=45, ha='right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No optimization history available")

## 8. Parameter Sensitivity Analysis

In [ ]:
# Analyze how L1 distance changes with parameter variations
theta_opt, M_opt, X_opt = optimizer.best_params

# Vary theta
theta_range = np.linspace(theta_opt - 5, theta_opt + 5, 50)
theta_distances = [optimizer.compute_L1_distance([t, M_opt, X_opt], data) 
                  for t in theta_range]

# Vary M
M_range = np.linspace(M_opt - 0.01, M_opt + 0.01, 50)
M_distances = [optimizer.compute_L1_distance([theta_opt, m, X_opt], data) 
              for m in M_range]

# Vary X
X_range = np.linspace(X_opt - 10, X_opt + 10, 50)
X_distances = [optimizer.compute_L1_distance([theta_opt, M_opt, x], data) 
              for x in X_range]

# Plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].plot(theta_range, theta_distances, linewidth=2)
axes[0].axvline(theta_opt, color='r', linestyle='--', label='Optimal')
axes[0].set_xlabel('θ (degrees)', fontsize=12)
axes[0].set_ylabel('L1 Distance', fontsize=12)
axes[0].set_title('Sensitivity to θ', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(M_range, M_distances, linewidth=2, color='green')
axes[1].axvline(M_opt, color='r', linestyle='--', label='Optimal')
axes[1].set_xlabel('M', fontsize=12)
axes[1].set_ylabel('L1 Distance', fontsize=12)
axes[1].set_title('Sensitivity to M', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(X_range, X_distances, linewidth=2, color='purple')
axes[2].axvline(X_opt, color='r', linestyle='--', label='Optimal')
axes[2].set_xlabel('X', fontsize=12)
axes[2].set_ylabel('L1 Distance', fontsize=12)
axes[2].set_title('Sensitivity to X', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Export Results

In [ ]:
# Save results
import json
from pathlib import Path

# Create results directory
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True)

# Save parameters
result['desmos_format'] = optimizer.generate_desmos_format()
with open(results_dir / 'parameters.json', 'w') as f:
    json.dump(result, f, indent=2)

print("Results saved to ../results/parameters.json")
print("\nDesmos Format (copy this):")
print(result['desmos_format'])

## 10. Summary

This notebook demonstrates:
1. Data loading and exploration
2. Parametric curve optimization
3. Result visualization and analysis
4. Residual analysis
5. Convergence monitoring
6. Parameter sensitivity analysis

The optimal parameters have been found and are ready for submission!